In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException

In [2]:
import pandas as pd
import numpy as np
import re
import time

In [3]:
def cafe_crawling2():
    description_state = False
    try:
        description_a_link = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt")
        description_a_link.click()
        description_state = True
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    

    image_div = soup.find('div', {'id': 'ibu_1'})
    if image_div:
        style = image_div.get('style')
        background_image_url = re.search(r'url\((.*?)\)', style).group(1)
    else:
        background_image_url = ""
    
    # reviews_div = soup.find('div', class_='dAsGb')
    # starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    # if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
    #     return None
    # visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    # blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    try:
        # 별점 요소 찾기
        star_rating_element = driver.find_element(By.CSS_SELECTOR, "div.dAsGb > span.PXMot.LXIwF")
        starring = float(star_rating_element.text.strip()[2:])
    except NoSuchElementException:
        # 별점 요소가 없는 경우
        starring = 0

    try:
        # 방문자 리뷰 요소 찾기
        visitor_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/visitor')]")
        visitor_reviews = int(visitor_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 방문자 리뷰 요소가 없는 경우
        visitor_reviews = 0
        
    try:
        # 블로그 리뷰 요소 찾기
        blog_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/ugc')]")
        blog_reviews = int(blog_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 블로그 리뷰 요소가 없는 경우
        blog_reviews = 0
    
    if visitor_reviews < 10:
        return None
    #PIbes > O8qbU > vV_z_ > zPfVt
    # description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else ""
    if description_state:
        description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt").text
    else:
        try:
            description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > div.xHaT3 >  span.zPfVt").text
        except:
            description = ""
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    
    keyword_state = False
    idx = 0
    while not keyword_state and idx < 5:
        try:
            keyword_list_link = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
            keyword_list_link.click()
            keyword_state = True
        except:
            for _ in range(5):
                keyword_tab_link.click()
                time.sleep(0.5)
        idx+=1
    
#     for _ in range(3):
#         try:
#             keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
#             keyword_list_link.click()
#             keyword_state = True
#         except:
#             keyword_tab_link.click()
    
    if not keyword_state:
        return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", "", "", background_image_url]

    while True:
        try:
            keyword_more_link = driver.find_element(By.CLASS_NAME, 'Tvx37')
            keyword_more_link.click()
        except Exception as ex:
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(driver.find_element(By.CSS_SELECTOR, "div._Wmab > em").text.replace(",", ""))

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number, background_image_url]

In [4]:
cafe_search_list= [
                   "동성로", "대구 중앙로", "교동", "경북대병원역",
                    "동대구역", "대구 평화시장", "안심역",
                    "대구 광장코아", "만평역", "대구 서구청",
                    "명덕역", "대구 앞산",
                    "경북대 북문", "대구 복현오거리", "동천동 카페", "칠곡네거리",
                    "범어네거리", "수성유원지", "시지", "지산",
                    "두류공원", "용산", "월배", "계명대"
                    ]

In [4]:
for cafe_name in ["대구", "경산", "황금동", "범어동"]:
    try:
        df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
                                    "menu", "keyword", 'keyword_number', 'image_link'])
        # WebDriver 초기화
        driver = webdriver.Chrome() # Chrome 드라이버 사용
        driver.get(f"https://map.naver.com/p/search/{cafe_name} 카페") # 네이버 지도 접속

        # 결과 로드 대기
        time.sleep(3) # 페이지 로드를 위해 잠시 대기


        while True:
            # searchIframe으로 컨텍스트 전환 + div scroll 내리기
            driver.switch_to.frame("searchIframe")
            scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
            for _ in range(5):
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)
                time.sleep(1)

            # 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
            cafe_list = scroll_div.find_elements(By.TAG_NAME, "li")
            cafe_size = len(cafe_list)


            for item in cafe_list:
                try:
                    place_bluelink = item.find_element(By.CLASS_NAME, "place_bluelink")
                    # 부모의 부모인 a 태그 찾기 및 클릭
                    parent_a_tag = place_bluelink.find_element(By.XPATH, "../..")
                    parent_a_tag.click()

                    time.sleep(3)

                    # 필요한 작업 수행 (예: 상세 정보 수집)
                    driver.switch_to.default_content()
                    driver.switch_to.frame("entryIframe")
                    cafe_row = cafe_crawling2()
                    if cafe_row:
                        df.loc[len(df)] = cafe_row
                        print(cafe_row)

                    # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전
                    driver.switch_to.default_content()
                    driver.switch_to.frame("searchIframe")

                except Exception as e:
                    print(e)
                    pass

            next_a_link = driver.find_elements(By.CLASS_NAME, "eUTV2")[1]
            aria_disabled = next_a_link.get_attribute("aria-disabled")
            # 'aria-disabled'가 'false'이면 a 태그 클릭
            if aria_disabled == "false":
                next_a_link.click()
            else:
                break

            # 원래 프레임으로 돌아감
            driver.switch_to.default_content()
    finally:
            df.to_csv(f"./new_cafe_data/{cafe_name}.csv", encoding='utf-8')


['크레페영 대구다사점', '크레페', 0, 265, 39, '대구 달성군 다사읍 달구벌대로 874 B동 1층 111호 크레페영', '11:30에 영업 시작', '0507-1447-6674', '', '', '', {'디저트가 맛있어요': 240, '친절해요': 192, '매장이 청결해요': 150, '특별한 메뉴가 있어요': 120, '가성비가 좋아요': 118, '음료가 맛있어요': 101, '커피가 맛있어요': 59, '인테리어가 멋져요': 51, '대화하기 좋아요': 20, '사진이 잘 나와요': 16, '음식이 맛있어요': 13, '메뉴 구성이 알차요': 12, '좌석이 편해요': 7, '주차하기 편해요': 6, '아이와 가기 좋아요': 5, '종류가 다양해요': 5, '아늑해요': 4, '음악이 좋아요': 3, '뷰가 좋아요': 3, '양이 많아요': 2, '포장이 깔끔해요': 2, '화장실이 깨끗해요': 2, '주문제작을 잘해줘요': 1, '음식이 빨리 나와요': 1, '비싼 만큼 가치있어요': 1, '집중하기 좋아요': 1}, 261, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230806_300%2F1691333146353UYLhF_PNG%2F%25B7%25CE%25B0%25ED.png"']
['돈까스 무한리필 돈신 동성로', '양식', 0, 281, 280, '대구 중구 남성로 40-1 2층', '11:30에 영업 시작', '0507-1341-8414', '', '', '', {'음식이 맛있어요': 192, '가성비가 좋아요': 143, '친절해요': 116, '양이 많아요': 109, '매장이 넓어요': 109, '매장이 청결해요': 100, '재료가 신선해요': 88, '특별한 메뉴가 있어요': 53, '단체모임 하기 좋아요': 40, '인테리어가 멋져요': 30, '혼밥하기 좋

['르미엘엠', '카페,디저트', 0, 96, 166, '대구 동구 신암남로23길 42', '11:30에 영업 시작', '0507-1328-1619', '', '', '', {'디저트가 맛있어요': 67, '커피가 맛있어요': 50, '선물하기 좋아요': 34, '인테리어가 멋져요': 27, '친절해요': 26, '음료가 맛있어요': 18, '가성비가 좋아요': 16, '특별한 메뉴가 있어요': 14, '매장이 청결해요': 5, '집중하기 좋아요': 4, '주차하기 편해요': 2, '사진이 잘 나와요': 2, '대화하기 좋아요': 2}, 89, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221007_162%2F1665131910632xIme2_JPEG%2F01_%25B8%25B6%25C4%25AB%25B7%25D5__%25285%2529.jpg"']
['롤러커피', '테이크아웃커피', 4.51, 1012, 1422, '대구 중구 달구벌대로414길 36', '08:00에 영업 시작', '053-253-7584', '', '', '', {'커피가 맛있어요': 555, '친절해요': 254, '음료가 맛있어요': 109, '가성비가 좋아요': 90, '인테리어가 멋져요': 88, '특별한 메뉴가 있어요': 67, '사진이 잘 나와요': 59, '매장이 청결해요': 49, '대화하기 좋아요': 35, '디저트가 맛있어요': 20, '뷰가 좋아요': 12, '화장실이 깨끗해요': 11, '좌석이 편해요': 7, '아늑해요': 6, '음악이 좋아요': 4, '집중하기 좋아요': 4, '컨셉이 독특해요': 2, '야외 공간이 멋져요': 2, '주차하기 편해요': 2, '차분한 분위기에요': 1, '주문제작을 잘해줘요': 1, '재료가 신선해요': 1, '반려동물과 가기 좋아요': 1, '차가 맛있어요': 1}, 

['초이스엠', '베이커리', 4.48, 3460, 843, '대구 동구 팔공산로 260', '10:00에 영업 시작', '0507-1485-7272', '', '', '', {'빵이 맛있어요': 1841, '주차하기 편해요': 593, '커피가 맛있어요': 517, '친절해요': 373, '특별한 메뉴가 있어요': 347, '매장이 청결해요': 285, '음료가 맛있어요': 253, '뷰가 좋아요': 238, '대화하기 좋아요': 227, '인테리어가 멋져요': 204, '좌석이 편해요': 134, '가성비가 좋아요': 109, '화장실이 깨끗해요': 80, '사진이 잘 나와요': 78, '집중하기 좋아요': 15, '매장이 넓어요': 13, '디저트가 맛있어요': 8, '건강한 맛이에요': 4, '단체모임 하기 좋아요': 3, '음식이 맛있어요': 2, '재료가 신선해요': 2, '비싼 만큼 가치있어요': 2, '종류가 다양해요': 2, '아늑해요': 1}, 2284, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230615_144%2F1686821474085qvjwK_JPEG%2FKakaoTalk_20230615_181342315.jpg"']
['민스크', '카페', 4.39, 1396, 675, '대구 남구 현충로6길 9-2', '11:00에 영업 시작', '053-656-6292', '', '', '', {'커피가 맛있어요': 436, '인테리어가 멋져요': 300, '디저트가 맛있어요': 241, '주차하기 편해요': 236, '음료가 맛있어요': 224, '뷰가 좋아요': 181, '대화하기 좋아요': 163, '매장이 청결해요': 149, '친절해요': 136, '사진이 잘 나와요': 114, '특별한 메뉴가 있어요': 75, '좌석이 편해요': 63, '화장실이 깨끗해요': 39, '가

['딥커피로스터스', '카페,디저트', 4.53, 756, 1186, '대구 중구 봉산문화2길 41 1층', '10:00에 영업 시작', '0507-1484-8340', '', '', '', {'커피가 맛있어요': 362, '인테리어가 멋져요': 114, '친절해요': 113, '특별한 메뉴가 있어요': 89, '가성비가 좋아요': 79, '대화하기 좋아요': 77, '음료가 맛있어요': 75, '매장이 청결해요': 60, '디저트가 맛있어요': 52, '사진이 잘 나와요': 43, '집중하기 좋아요': 22, '뷰가 좋아요': 13, '좌석이 편해요': 11, '화장실이 깨끗해요': 5, '차분한 분위기에요': 2, '음악이 좋아요': 2, '종류가 다양해요': 2, '아늑해요': 1, '주차하기 편해요': 1, '매장이 넓어요': 1}, 453, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20201222_61%2F16086101773867psbu_JPEG%2FKIJlei8S8v85I550Ytau4bMZ.jpeg.jpg"']
['팜테이블', '카페,디저트', 4.46, 2564, 788, '대구 중구 동성로6길 75 1층', '11:00에 영업 시작', '0507-1329-7436', '', '', '', {'디저트가 맛있어요': 1729, '커피가 맛있어요': 840, '음료가 맛있어요': 806, '친절해요': 699, '인테리어가 멋져요': 662, '특별한 메뉴가 있어요': 660, '매장이 청결해요': 572, '대화하기 좋아요': 364, '사진이 잘 나와요': 311, '좌석이 편해요': 91, '가성비가 좋아요': 83, '화장실이 깨끗해요': 53, '뷰가 좋아요': 53, '매장이 넓어요': 24, '집중하기 좋아요': 22, '음식이 맛있어요': 18, '빵이 맛있어요': 12, '

['도우맨도넛', '도넛', 4.72, 460, 455, '대구 동구 동부로30길 3-5 DOUGHMAN', '12:00에 영업 시작', '0507-1305-1281', '', '', '', {'디저트가 맛있어요': 233, '친절해요': 116, '커피가 맛있어요': 102, '특별한 메뉴가 있어요': 85, '인테리어가 멋져요': 62, '음료가 맛있어요': 54, '매장이 청결해요': 50, '대화하기 좋아요': 43, '좌석이 편해요': 24, '화장실이 깨끗해요': 22, '사진이 잘 나와요': 20, '가성비가 좋아요': 13, '집중하기 좋아요': 9, '뷰가 좋아요': 6, '아늑해요': 4, '차분한 분위기에요': 3, '빵이 맛있어요': 1, '음식이 맛있어요': 1, '포장이 깔끔해요': 1, '비싼 만큼 가치있어요': 1}, 313, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230312_4%2F167859134164234yo7_JPEG%2FA190A8AE-A5D5-4246-B15B-5259AD8333CB.jpeg"']
['두낫디스터브 동대구점', '카페,디저트', 4.7, 883, 290, '대구 동구 동부로30길 9 나동 1층', '10:00에 영업 시작', '0507-1394-5125', '', '', '', {'커피가 맛있어요': 340, '인테리어가 멋져요': 231, '음료가 맛있어요': 198, '친절해요': 175, '대화하기 좋아요': 159, '사진이 잘 나와요': 152, '디저트가 맛있어요': 141, '매장이 청결해요': 122, '좌석이 편해요': 68, '특별한 메뉴가 있어요': 61, '뷰가 좋아요': 55, '가성비가 좋아요': 50, '집중하기 좋아요': 41, '화장실이 깨끗해요': 17, '주차하기 편해요': 7, '반려동물과 가기 좋아요': 

...

['히스', '카페', 0, 145, 186, '대구 수성구 청호로85길 9 1층', '10:30에 영업 시작', '0507-1319-7240', '', '', '', {'커피가 맛있어요': 54, '친절해요': 36, '인테리어가 멋져요': 19, '음료가 맛있어요': 16, '집중하기 좋아요': 16, '대화하기 좋아요': 14, '디저트가 맛있어요': 12, '매장이 청결해요': 12, '특별한 메뉴가 있어요': 7, '좌석이 편해요': 3, '차분한 분위기에요': 2, '뷰가 좋아요': 2, '메뉴 구성이 알차요': 1, '주차하기 편해요': 1, '사진이 잘 나와요': 1, '가성비가 좋아요': 1, '화장실이 깨끗해요': 1}, 74, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20190807_82%2F1565159059067R3ArL_JPEG%2Fi-u_L1VfyPDtxEYjOP4GbDzh.jpeg.jpg"']
['바게리', '브런치', 0, 275, 445, '대구 수성구 달구벌대로489길 23 카페 바게리', '08:30에 영업 시작', '0507-1358-6943', '', '', '', {'음식이 맛있어요': 167, '인테리어가 멋져요': 124, '친절해요': 89, '매장이 넓어요': 84, '특별한 메뉴가 있어요': 72, '매장이 청결해요': 72, '재료가 신선해요': 60, '뷰가 좋아요': 22, '양이 많아요': 19, '주차하기 편해요': 16, '화장실이 깨끗해요': 12, '디저트가 맛있어요': 11, '가성비가 좋아요': 10, '단체모임 하기 좋아요': 10, '커피가 맛있어요': 8, '특별한 날 가기 좋아요': 8, '사진이 잘 나와요': 4, '대화하기 좋아요': 4, '차분한 분위기에요': 3, '음료가 맛있어요': 3, '컨셉이 독특해요': 2, '야

['도헤과자점', '카페,디저트', 0, 334, 133, '대구 수성구 범어로24길 20 1층', '11:00에 영업 시작', '0507-1449-0051', '', '', '', {'디저트가 맛있어요': 304, '친절해요': 157, '인테리어가 멋져요': 152, '매장이 청결해요': 147, '음료가 맛있어요': 133, '커피가 맛있어요': 132, '특별한 메뉴가 있어요': 106, '사진이 잘 나와요': 74, '대화하기 좋아요': 68, '가성비가 좋아요': 25, '집중하기 좋아요': 16, '화장실이 깨끗해요': 16, '좌석이 편해요': 13, '뷰가 좋아요': 13, '차분한 분위기에요': 10, '아늑해요': 10, '종류가 다양해요': 5, '메뉴 구성이 알차요': 4, '음악이 좋아요': 4, '빵이 맛있어요': 3, '음식이 맛있어요': 3, '선물하기 좋아요': 3, '포장이 깔끔해요': 3, '비싼 만큼 가치있어요': 3, '주문제작을 잘해줘요': 1, '룸이 잘 되어있어요': 1}, 332, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230808_132%2F1691485575396Gio0e_JPEG%2F53BB2CC6-7806-4651-8070-50D86ADFA847.jpeg"']
['카페더시즈', '카페,디저트', 4.7, 153, 153, '대구 수성구 달구벌대로492길 49 1층', '09:00에 영업 시작', '0507-1333-3427', '', '', '', {'인테리어가 멋져요': 65, '커피가 맛있어요': 53, '디저트가 맛있어요': 45, '사진이 잘 나와요': 40, '음료가 맛있어요': 33, '친절해요': 33, '특별한 메뉴가 있어요': 28, '컨셉이 독특해요': 25, '매장이 청결해요': 22, '대화하기 좋아요': 17, '좌석이 

['파더스베이글 대구범어점', '카페', 4.65, 417, 61, '대구 수성구 상록로 15 범어 센트럴푸르지오상가 1층 105호', '09:00에 영업 시작', '0507-1488-3303', '', '', '', {'빵이 맛있어요': 115, '매장이 청결해요': 96, '친절해요': 89, '디저트가 맛있어요': 77, '재료가 신선해요': 47, '커피가 맛있어요': 42, '종류가 다양해요': 37, '특별한 메뉴가 있어요': 26, '인테리어가 멋져요': 24, '가성비가 좋아요': 16, '아늑해요': 15, '음료가 맛있어요': 15, '좌석이 편해요': 8, '주차하기 편해요': 8, '사진이 잘 나와요': 7, '대화하기 좋아요': 7, '집중하기 좋아요': 3, '컨셉이 독특해요': 2, '음악이 좋아요': 2, '건강한 맛이에요': 1, '화장실이 깨끗해요': 1}, 287, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210108_241%2F1610082821384XAYXF_JPEG%2F55vcxS0ZOf-odv149YF65nud.jpeg.jpg"']
['노바디커피클럽', '카페', 0, 168, 142, '대구 수성구 청솔로14길 16 1층', '08:00에 영업 시작', '0507-1486-0714', '', '', '', {'커피가 맛있어요': 144, '친절해요': 96, '인테리어가 멋져요': 86, '사진이 잘 나와요': 76, '디저트가 맛있어요': 66, '음료가 맛있어요': 53, '가성비가 좋아요': 29, '매장이 청결해요': 27, '대화하기 좋아요': 10, '특별한 메뉴가 있어요': 6, '집중하기 좋아요': 6, '뷰가 좋아요': 5, '아늑해요': 2, '차분한 분위기에요': 1, '컨셉이 독특해요': 1, '주차하기 편해요': 1, '화장실이 깨끗

['롤링핀 범어점', '베이커리', 4.53, 693, 67, '대구 수성구 달구벌대로496길 50', '07:30에 영업 시작', '053-752-9955', '', '', '', {'빵이 맛있어요': 353, '커피가 맛있어요': 116, '친절해요': 91, '특별한 메뉴가 있어요': 55, '매장이 청결해요': 53, '음료가 맛있어요': 51, '대화하기 좋아요': 40, '가성비가 좋아요': 19, '인테리어가 멋져요': 17, '좌석이 편해요': 14, '주차하기 편해요': 9, '사진이 잘 나와요': 8, '집중하기 좋아요': 8, '화장실이 깨끗해요': 8, '뷰가 좋아요': 8, '디저트가 맛있어요': 1, '재료가 신선해요': 1, '종류가 다양해요': 1}, 450, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200707_6%2F1594108879108UANQP_JPEG%2Fb37xBkazuaHTXuNPPXWmsY7P.jpg"']
['공차 범어로데오점', '차', 4.56, 1252, 15, '대구 수성구 달구벌대로 2513', '09:00에 영업 시작', '0507-1339-1055', '', '', '', {'음료가 맛있어요': 461, '차가 맛있어요': 258, '친절해요': 137, '매장이 청결해요': 73, '특별한 메뉴가 있어요': 65, '집중하기 좋아요': 62, '디저트가 맛있어요': 48, '좌석이 편해요': 45, '대화하기 좋아요': 38, '가성비가 좋아요': 32, '인테리어가 멋져요': 21, '주차하기 편해요': 7, '뷰가 좋아요': 7, '화장실이 깨끗해요': 5, '사진이 잘 나와요': 4, '종류가 다양해요': 2, '커피가 맛있어요': 1}, 765, '"https://search.pstatic.net/common/?autoRotate=t

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x00007FF7B51582B2+55298]
	(No symbol) [0x00007FF7B50C5E02]
	(No symbol) [0x00007FF7B4F805AB]
	(No symbol) [0x00007FF7B4F6E19B]
	(No symbol) [0x00007FF7B4F6DDF0]
	(No symbol) [0x00007FF7B4F824A1]
	(No symbol) [0x00007FF7B4FFA18F]
	(No symbol) [0x00007FF7B4FE1E83]
	(No symbol) [0x00007FF7B4FB670A]
	(No symbol) [0x00007FF7B4FB7964]
	GetHandleVerifier [0x00007FF7B54D0AAB+3694587]
	GetHandleVerifier [0x00007FF7B552728E+4048862]
	GetHandleVerifier [0x00007FF7B551F173+4015811]
	GetHandleVerifier [0x00007FF7B51F47D6+695590]
	(No symbol) [0x00007FF7B50D0CE8]
	(No symbol) [0x00007FF7B50CCF34]
	(No symbol) [0x00007FF7B50CD062]
	(No symbol) [0x00007FF7B50BD3A3]
	BaseThreadInitThunk [0x00007FFE372D257D+29]
	RtlUserThreadStart [0x00007FFE38B0AA78+40]
